In [1]:
import mido

In [2]:
mid = mido.MidiFile('up.mid')

In [8]:
msg = mid.tracks[0][12]

In [31]:
mid

<midi file 'up.mid' type 1, 2 tracks, 1643 messages>

In [27]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    tempo = 0
    for msg in track:
        print(msg)
#         if msg.is_meta:
#             if msg.type == 'set_tempo':
#                 tempo = msg.dict()['tempo']
#                 print(tempo)

Track 0: 
<meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='F' time=0>
<meta message set_tempo tempo=342857 time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=2
control_change channel=0 control=10 value=64 time=2
control_change channel=0 control=91 value=30 time=2
control_change channel=0 control=93 value=30 time=2
note_on channel=0 note=77 velocity=80 time=5032
note_on channel=0 note=77 velocity=0 time=239
note_on channel=0 note=81 velocity=80 time=1
note_on channel=0 note=81 velocity=0 time=239
note_on channel=0 note=77 velocity=80 time=1
note_on channel=0 note=77 velocity=0 time=239
note_on channel=0 note=76 velocity=80 time=1
note_on channel=0 note=76 velocity=0 time=1919
note_on channel=0 note=77 velocity=80 time=241
note_on channel=0 note=77 velocity=0 time=239
note_on channel=0 note=81 velocity=80 time=1
note_on channel=0 note=81 veloci

note_on channel=0 note=53 velocity=64 time=0
note_on channel=0 note=60 velocity=0 time=2879
note_on channel=0 note=57 velocity=0 time=0
note_on channel=0 note=53 velocity=0 time=0
note_on channel=0 note=60 velocity=64 time=1
note_on channel=0 note=55 velocity=64 time=0
note_on channel=0 note=52 velocity=64 time=0
note_on channel=0 note=60 velocity=0 time=2879
note_on channel=0 note=55 velocity=0 time=0
note_on channel=0 note=52 velocity=0 time=0
note_on channel=0 note=77 velocity=33 time=1
note_on channel=0 note=69 velocity=33 time=0
note_on channel=0 note=77 velocity=0 time=1439
note_on channel=0 note=69 velocity=0 time=0
note_on channel=0 note=76 velocity=33 time=1
note_on channel=0 note=69 velocity=33 time=0
note_on channel=0 note=76 velocity=0 time=1439
note_on channel=0 note=69 velocity=0 time=0
note_on channel=0 note=77 velocity=33 time=1
note_on channel=0 note=69 velocity=33 time=0
note_on channel=0 note=77 velocity=0 time=1439
note_on channel=0 note=69 velocity=0 time=0
note_on

In [17]:
float(5032)/239

21.05439330543933